# Extract the First X Theme Tunes

This differs from the other notebook `Extract Title Music` in that it expects to find multiple theme tunes in the x64 trace file. It will write out sheet music for each separately.

We're going to extract an example of the procedurally generated title music and convert it to sheet music.

Going to do this by tracing all writes to `D401 - D415` and storing them in a log file. These are the Vice monitor commands for doing this:
```
log on                               
logname "IridisAlphaTitleMusic.txt"
tr store d401 d415                   
```

We'll then extract the notes as they are written to the log file.

In [183]:
"""
From the C64 Programmer's reference manual. These are also the notes used by 
Iridis Alpha in:

                    ;      C   C#  D   D#  E   F   F#  G   G#  A   A#  B      
titleMusicHiBytes   .BYTE $08,$08,$09,$09,$0A,$0B,$0B,$0C,$0D,$0E,$0E,$0F  ; 4
                    .BYTE $10,$11,$12,$13,$15,$16,$17,$19,$1A,$1C,$1D,$1F  ; 5
                    .BYTE $21,$23,$25,$27,$2A,$2C,$2F,$32,$35,$38,$3B,$3F  ; 6
                    .BYTE $43,$47,$4B,$4F,$54,$59,$5E,$64,$6A,$70,$77,$7E  ; 7
                    .BYTE $86,$8E,$96,$9F,$A8,$B3,$BD,$C8,$D4,$E1,$EE,$FD  ; 8
                                                                              
                    ;      C   C#  D   D#  E   F   F#  G   G#  A   A#  B      
titleMusicLowBytes  .BYTE $61,$E1,$68,$F7,$8F,$30,$DA,$8F,$4E,$18,$EF,$D2  ; 4
                    .BYTE $C3,$C3,$D1,$EF,$1F,$60,$B5,$1E,$9C,$31,$DF,$A5  ; 5
                    .BYTE $87,$86,$A2,$DF,$3E,$C1,$6B,$3C,$39,$63,$BE,$4B  ; 6
                    .BYTE $0F,$0C,$45,$BF,$7D,$83,$D6,$79,$73,$C7,$7C,$97  ; 7
                    .BYTE $1E,$18,$8B,$7E,$FA,$06,$AC,$F3,$E6,$8F,$F8,$2E  ; 8


"""
# We're going to extract the values from here, convert to hex, and create a 
# dict mapping hex values to musical note.

raw_notes_table = """
+-----------------------------+-----------------------------------------+
|        MUSICAL NOTE         |             OSCILLATOR FREQ             |
+-------------+---------------+-------------+-------------+-------------+
|     NOTE    |    OCTAVE     |   DECIMAL   |      HI     |     LOW     |
+-------------+---------------+-------------+-------------+-------------+
|       0     |      C-0      |     268     |       1     |      12     | 
|       1     |      C#-0     |     284     |       1     |      28     | 
|       2     |      D-0      |     301     |       1     |      45     | 
|       3     |      D#-0     |     318     |       1     |      62     | 
|       4     |      E-0      |     337     |       1     |      81     | 
|       5     |      F-0      |     358     |       1     |     102     | 
|       6     |      F#-0     |     379     |       1     |     123     | 
|       7     |      G-0      |     401     |       1     |     145     | 
|       8     |      G#-0     |     425     |       1     |     169     | 
|       9     |      A-0      |     451     |       1     |     195     | 
|      10     |      A#-0     |     477     |       1     |     221     | 
|      11     |      B-0      |     506     |       1     |     250     | 
|      16     |      C-1      |     536     |       2     |      24     | 
|      17     |      C#-1     |     568     |       2     |      56     | 
|      18     |      D-1      |     602     |       2     |      90     | 
|      19     |      D#-1     |     637     |       2     |     125     | 
|      20     |      E-1      |     675     |       2     |     163     | 
|      21     |      F-1      |     716     |       2     |     204     | 
|      22     |      F#-1     |     758     |       2     |     246     | 
|      23     |      G-1      |     803     |       3     |      35     | 
|      24     |      G#-1     |     851     |       3     |      83     | 
|      25     |      A-1      |     902     |       3     |     134     | 
|      26     |      A#-1     |     955     |       3     |     187     | 
|      27     |      B-1      |    1012     |       3     |     244     | 
|      32     |      C-2      |    1072     |       4     |      48     | 
|      33     |      C#-2     |     1136    |       4     |     112     | 
|      34     |      D-2      |     1204    |       4     |     180     | 
|      35     |      D#-2     |     1275    |       4     |     251     | 
|      36     |      E-2      |     1351    |       5     |      71     | 
|      37     |      F-2      |     1432    |       5     |     152     | 
|      38     |      F#-2     |     1517    |       5     |     237     | 
|      39     |      G-2      |     1607    |       6     |      71     | 
|      40     |      G#-2     |     1703    |       6     |     167     | 
|      41     |      A-2      |     1804    |       7     |      12     | 
|      42     |      A#-2     |     1911    |       7     |     119     | 
|      43     |      B-2      |     2025    |       7     |     233     | 
|      48     |      C-3      |     2145    |       8     |      97     |
|      49     |      C#-3     |     2273    |       8     |     225     |
|      50     |      D-3      |     2408    |       9     |     104     |
|      51     |      D#-3     |     2551    |       9     |     247     |
|      52     |      E-3      |     2703    |      10     |     143     |
|      53     |      F-3      |     2864    |      11     |      48     |
|      54     |      F#-3     |     3034    |      11     |     218     |
|      55     |      G-3      |     3215    |      12     |     143     |
|      56     |      G#-3     |     3406    |      13     |      78     |
|      57     |      A-3      |     3608    |      14     |      24     |
|      58     |      A#-3     |     3823    |      14     |     239     |
|      59     |      B-3      |     4050    |      15     |     210     |
|      64     |      C-4      |     4291    |      16     |     195     |
|      65     |      C#-4     |     4547    |      17     |     195     |
|      66     |      D-4      |     4817    |      18     |     209     |
|      67     |      D#-4     |     5103    |      19     |     239     |
|      68     |      E-4      |     5407    |      21     |      31     |
|      69     |      F-4      |     5728    |      22     |      96     |
|      70     |      F#-4     |     6069    |      23     |     181     |
|      71     |      G-4      |     6430    |      25     |      30     |
|      72     |      G#-4     |     6812    |      26     |     156     |
|      73     |      A-4      |     7217    |      28     |      49     |
|      74     |      A#-4     |     7647    |      29     |     223     |
|      75     |      B-4      |     8101    |      31     |     165     |
|      80     |      C-5      |     8583    |      33     |     135     |
|      81     |      C#-5     |     9094    |      35     |     134     |
|      82     |      D-5      |     9634    |      37     |     162     |
|      83     |      D#-5     |    10207    |      39     |     223     |
|      84     |      E-5      |    10814    |      42     |      62     |
|      85     |      F-5      |    11457    |      44     |     193     |
|      86     |      F#-5     |    12139    |      47     |     107     |
|      87     |      G-5      |    12860    |      50     |      60     |
|      88     |      G#-5     |    13625    |      53     |      57     |
|      89     |      A-5      |    14435    |      56     |      99     |
|      90     |      A#-5     |    15294    |      59     |     190     |
|      91     |      B-5      |    16203    |      63     |      75     |
|      96     |      C-6      |    17167    |      67     |      15     |
|      97     |      C#-6     |    18188    |      71     |      12     |
|      98     |      D-6      |    19269    |      75     |      69     |
|      99     |      D#-6     |    20415    |      79     |     191     |
|     100     |      E-6      |    21629    |      84     |     125     |
|     101     |      F-6      |    22915    |      89     |     131     |
|     102     |      F#-6     |    24278    |      94     |     214     |
|     103     |      G-6      |    25721    |     100     |     121     |
|     104     |      G#-6     |    27251    |     106     |     115     |
|     105     |      A-6      |    28871    |     112     |     199     |
|     106     |      A#-6     |    30588    |     119     |     124     |
|     107     |      B-6      |    32407    |     126     |     151     |
|     112     |      C-7      |    34334    |     134     |      30     |
|     113     |      C#-7     |    36376    |     142     |      24     |
|     114     |      D-7      |    38539    |     150     |     139     |
|     115     |      D#-7     |    40830    |     159     |     126     |
|     116     |      E-7      |    43258    |     168     |     250     |
|     117     |      F-7      |    45830    |     179     |       6     |
|     118     |      F#-7     |    48556    |     189     |     172     |
|     119     |      G-7      |    51443    |     200     |     243     |
|     120     |      G#-7     |    54502    |     212     |     230     |
|     121     |      A-7      |    57743    |     225     |     143     |
|     122     |      A#-7     |    61176    |     238     |     248     |
|     123     |      B-7      |    64814    |     253     |      46     |
+-------------+---------------+-------------+-------------+-------------+
"""

Create a dictionary that maps the hi/lo bytes in the table above to the corresponding note.

In [184]:
sid_to_note = {}
for n in raw_notes_table.split('\n'):
    nt = n[21:25].strip()
    if not n[50:54].strip().isdigit():
        continue
    hi = hex(int(n[50:54].strip()))[2:].upper().zfill(2)
    lo = hex(int(n[64:68].strip()))[2:].upper().zfill(2)
    sid_to_note[hi+lo] = nt


Extract the notes played from the Vice X64 log file. We store them as a list with each element like `("Voice1", "C-3")`, i.e. (voice, note). We get the notes by mapping from the hi/lo bytes logged by x64 to the notes they indicated using `sid_to_note`, which we created above.

In [185]:
log_file = "IridisAlphaTitleMusic-Multiple.txt"
input_file = open(log_file,'r')
tunes_raw = []
notes = []
for l in input_file.readlines():
    def addNote(voice, hi, lo):
        global notes
        note = hi+lo
        notes += [(voice, sid_to_note[note])]
    
    if any([s in l for s in [",Y",",X"]]):
        if notes:
            tunes_raw += [notes]
        notes = []
        continue

    if any([s in l for s in ["D400","D407", "D40E"]]):
        lo = l[40:42]

    for v,r in [("Voice1","D401"), ("Voice2","D408"),
                ("Voice3","D40F")]:
        if r not in l:
            continue
        hi = l[40:42]
        addNote(v, hi, lo)


Split into 3 arrays of the same length, an array for each voice. 

In [186]:
def generateVoiceData(notes):
    voice3 = [n for v,n in notes if v == "Voice3"]
    array_len = len(voice3)

    voice1 = ['']  * array_len
    voice2 = ['']  * array_len
    v3_pos = 0
    for (v,n) in notes:
        if v == "Voice3":
            v3_pos += 1
            continue
        if v == "Voice2":
            voice2[v3_pos] = n
        if v == "Voice1":
            voice1[v3_pos] = n
    return (voice1,voice2,voice3)

tunes_voicedata = []
for tune in tunes_raw:
    tunes_voicedata += [generateVoiceData(tune)]
len(tunes_voicedata[2][1])

256

In [187]:
def convertToABC(index, note):
    # If it starts with nothing, populate a 'rest'.
    if not index and not note:
        return 'z'
    if '-' not in note:
        return note
    n,o = note.split('-')
    if "#" in n:
        n = "^" + n[:1]
    if o == "3":
        return n+","
    if o == "4":
        return n
    if o == "5":
        return n.lower()
    if o == "6":
        return n.lower() + "'"
    if o == "7":
        return n.lower() + "''"
    return n


Convert the arrays to `abc` notation.

In [188]:

def createABCLines(voice):
    pi = 0
    voice_abc = [convertToABC(i,n) for i,n in enumerate(voice)]
    for i,v in enumerate(voice_abc):
        if not v or not i:
            continue
        pl = i - pi
        voice_abc[pi] = voice_abc[pi] + str(pl)
        pi = i
    return voice_abc + [":|"] # Add the end tune notation


In [189]:
tunes = []
for voice1,voice2,voice3 in tunes_voicedata:
    tunes += [(createABCLines(voice1),
               createABCLines(voice2),
               createABCLines(voice3))]


Segment the `abc` notation and write to `abc` files.

In [190]:
abc_filename_prefix = f"../src/music/" 

def writeMusicPage(tune, title, pg_no, start, length):
    voice1_abc, voice2_abc, voice3_abc = tune
    abc_filename = f"title_no_{title+1}_page_{pg_no}.abc"
    output_file = open(abc_filename_prefix + abc_filename, 'w')
    header = f"""
%abc-2.2
%%pagewidth 30cm
%%header "Example		page: $P"
%%footer "	$T"
%%gutter .5cm
%%barsperstaff 16
%%titleformat R-P-Q-T C1 O1, T+T N1
%%composerspace 0
X: 2 % start of header
T:Iridis Alpha Title Theme
T:{title+1} of 100,000,000,000,000
C: (Sid.)
O: Jeff Minter
R:Procedurally Generated
L: 1/8
K: D % scale: C major
"""
    output_file.write(header)
    for i,voice_abc in enumerate([voice1_abc, voice2_abc, voice3_abc]):
        with_bars = ' '.join([n if (i+1) % 4 else n + ' |' 
                          for i,n in enumerate(voice_abc[start:start+length])])
        if i == 2:
            with_bars = with_bars.replace(' ','')
        output_file.write(f"V:{i+1} name=\"Voice {i+1}\"\n")
        output_file.write(f"{with_bars}\n")
    output_file.close()
    return abc_filename

abc_filenames = []
for title, tune in enumerate(tunes):
    abc_filenames += [writeMusicPage(tune, title, 1, 0, len(tune[0]))]
    #abc_filenames += [writeMusicPage(tune, title, 2, 128, 128)]


Convert the `abc` files to `svg`.

In [194]:
import subprocess

for abc_filename in abc_filenames:
    abc_file = abc_filename_prefix + abc_filename
    svg_file = abc_filename_prefix + f"{abc_filename[:-4]}.svg"
    png_file = abc_filename_prefix + f"{abc_filename[:-4]}.png"
    subprocess.run(["abcm2ps", "-g", abc_file, "-O", svg_file])
    #print(abc_file, svg_file,png_file)
    

 Convert all `svg` files in `src/music` to `png`.

In [195]:
from cairosvg import svg2png
import os

for fl in os.listdir(abc_filename_prefix):
    if not fl.endswith("svg"):
        continue
    svg_file = abc_filename_prefix + fl
    png_file = abc_filename_prefix + f"{fl[:-4]}.png"
    svg2png(url=svg_file, write_to=png_file)

